# Clasterization

--- 
Author: Anatoliy Durkin

Updated: 25.04.2025

---
В ноутбуке рассмотрены методы кластеризации

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

Что такое кластеризация?

Чем кластеризация отличается от классификации?

Для каких задач используется кластеризация?

In [ ]:
from sklearn.datasets import make_blobs

In [ ]:
# Генерируем синтетические данные
X, y = make_blobs(n_samples=300, centers=3, random_state=42)
plt.scatter(X[:, 0], X[:, 1], s=50)
plt.title("Данные для кластеризации")

## K-Means

In [ ]:
peng = pd.read_csv('penguins.csv')

In [ ]:
peng = peng[peng['sex']!='.']
peng = peng[(peng['flipper_length_mm']<1000) & (peng['flipper_length_mm']>100)]
peng = peng.dropna().reset_index(drop=True)
peng = pd.get_dummies(peng, drop_first=True)

In [ ]:
peng.head()

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(peng)

In [ ]:
plt.scatter(peng['culmen_length_mm'], peng['culmen_depth_mm'], c=clusters, s=50, cmap='viridis')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], c='red', s=200, marker='X')
plt.title("K-Means: Результат кластеризации")
plt.show()

In [ ]:
peng2 = peng.copy()
peng2['cl'] = clusters

In [ ]:
sns.pairplot(peng2, hue='cl')

In [ ]:
scaler = StandardScaler()
peng = pd.DataFrame(scaler.fit_transform(peng), columns=peng.columns)

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(peng)

In [ ]:
plt.scatter(peng['culmen_length_mm'], peng['culmen_depth_mm'], c=clusters, s=50, cmap='viridis')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], c='red', s=200, marker='X')
plt.title("K-Means: Результат кластеризации")
plt.show()

In [ ]:
peng2 = peng.copy()
peng2['cl'] = clusters

In [ ]:
sns.pairplot(peng2, hue='cl')

Что не так с кластеризацией в этот раз? Как это можно исправить? Попробуйте внести изменения,чтобы кластеризация сработала лучше.

In [ ]:
# Ваш код
...

### Метод локтя

Так же, как и для выбора оптимального числа компонент при снижении размерности, есть метод локтя для определения оптимального числа кластеров алгоритма KMeans. 

In [ ]:
inertia = []
for k in range(2, 7):
    kmeans = KMeans(n_clusters=k, random_state=42).fit(peng)
    inertia.append(kmeans.inertia_)  # Сумма квадратов расстояний до центроидов

plt.plot(range(2, 7), inertia, marker='o')
plt.xlabel("Число кластеров")
plt.ylabel("Inertia")
plt.title("Метод локтя")
plt.show()

Почему `inertia` уменьшается с числом кластеров?

Выбирать стоит число кластеров, где происходит излом, если он резкий. Если он мягкий, то стоит выбирать из близких значений.

Метрики:
- `silhouette_score`: оценка компактности и разделимости кластеров.
- `adjusted_rand_score`: сравнение с истинными метками (если есть).

In [ ]:
from sklearn.metrics import silhouette_score, adjusted_rand_score

In [ ]:
score_kmeans = silhouette_score(peng, clusters)
print(f"Silhouette Score (K-Means): {score_kmeans:.2f}")

Эту метрику также удобно использовать для выбора количества кластеров, но тут нужно брать число кластеров с максимальным значением метрики.

In [ ]:
silhouette = []
for k in range(2, 7):
    clusters = KMeans(n_clusters=k, random_state=42).fit_predict(peng)
    silhouette.append(silhouette_score(peng, clusters))

plt.plot(range(2, 7), silhouette, marker='o')
plt.xlabel("Число кластеров")
plt.ylabel("Silhouette")
plt.title("График силуэт")
plt.show()

Загрузим еще один датасет с данными по клиентам.

In [ ]:
cust = pd.read_csv('customer_segmentation.csv')

In [ ]:
cust.info()

In [ ]:
cust = cust.drop(['Dt_Customer'], axis=1)
cust['Education'] = LabelEncoder().fit_transform(cust['Education'])
cust['Marital_Status'] = LabelEncoder().fit_transform(cust['Marital_Status'])

cust = pd.DataFrame(StandardScaler().fit_transform(cust), columns=cust.columns).dropna()

In [ ]:
cust

In [ ]:
cust_pca = PCA(n_components=2).fit_transform(cust)

plt.scatter(cust_pca[:, 0], cust_pca[:, 1])
plt.title("Снижение размерности PCA")
plt.show()

Воспользуйтесь методом локтя,чтобы определить, на сколько кластеров можно раделить эти данные. Также можно построить график силуэт.

In [ ]:
# Ваш код
...

Возпользуйтесь методом KMeans для кластеризации.

In [ ]:
# Ваш код
...

In [ ]:
# Визуализация 
plt.scatter(cust_pca[:, 0], cust_pca[:, 1], c=clusters, cmap='viridis')
plt.title("Кластеризация KMeans")
plt.show()

## Иерархическая кластеризация

Агломеративный подход: каждая точка — отдельный кластер, затем объединяем ближайшие кластеры.

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

In [ ]:
Z = linkage(peng, method='ward')
plt.figure(figsize=(10, 5))
dendrogram(Z)
plt.title("Дендрограмма")
plt.show()

Дендрограмма показывает, как отдельные элементы объединяются в кластеры. В итоге получается один кластер - весь набор данных. Но, прерывая процесс объединения раньше, можно получить желаемое количество кластеров. Это и делает следующий метод.

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
agg = AgglomerativeClustering(n_clusters=3, linkage='ward')
clusters_agg = agg.fit_predict(peng)

In [ ]:
plt.scatter(peng['culmen_length_mm'], peng['culmen_depth_mm'], c=clusters_agg, cmap='viridis')
plt.title("Иерархическая кластеризация")
plt.show()

Все ли хорошо с этой кластеризацией? Что можно исправить?

Теперь постройте дендрограмму для данных по клиентам.

In [ ]:
# Ваш код
...

Какое количество кластеров будет хорошим вариантом? Можете оценить, как метод разделит данные при разном количестве кластеров.

In [ ]:
agg = AgglomerativeClustering(n_clusters=6, linkage='ward')
clusters_agg = agg.fit_predict(cust)

In [ ]:
plt.scatter(cust_pca[:, 0], cust_pca[:, 1], c=clusters_agg, cmap='viridis')
plt.title("Иерархическая кластеризация")
plt.show()

## DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons

In [ ]:
X_moons, _ = make_moons(n_samples=300, noise=0.1, random_state=42)

In [ ]:
plt.scatter(X_moons[:, 0], X_moons[:, 1])
plt.title("Синтетические данные")
plt.show()

In [ ]:
dbscan = DBSCAN(eps=0.2, min_samples=5)
clusters_db = dbscan.fit_predict(X_moons)

In [ ]:
plt.scatter(X_moons[:, 0], X_moons[:, 1], c=clusters_db, cmap='viridis')
plt.title("DBSCAN: Результат кластеризации")
plt.show()

Еще одно отличное применение этого метода - обнаружение шумов. Обратите внимание, как при снижении расстояния между соседями отстоящие чуть дальше точки могут отделяться от основных кластеров. Это как раз процесс выделения шумов.

In [ ]:
dbscan = DBSCAN(eps=0.15, min_samples=5)
clusters_db = dbscan.fit_predict(X_moons)

In [ ]:
plt.scatter(X_moons[:, 0], X_moons[:, 1], c=clusters_db, cmap='viridis')
plt.title("DBSCAN: Результат кластеризации")
plt.show()

А как с этими же данными справится KMeans?

In [ ]:
# Ваш код
...

Посмотрим, как DBSCAN разделит данные по пингвинам, и возможноли получить на этих данных хорошее разделение.

In [ ]:
dbscan = DBSCAN(eps=1, min_samples=5)
clusters_db = dbscan.fit_predict(peng.dropna())

In [ ]:
plt.scatter(peng['culmen_length_mm'], peng['culmen_depth_mm'], c=clusters_db, cmap='viridis')
plt.title("DBSCAN: Результат кластеризации")
plt.show()